In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [2]:
# Load the data
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

# Assign X and Y from training data
X_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"].values

# Normalize the data for better performance
X_train = X_train / 255.0
test = test / 255.0

# Reshape image in 3 dimensions (height = 28px, width = 28px , channel = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)

In [3]:
model_count = 10
model = [0]*model_count
# ENSEMBLE of models
for i in range(model_count):
    model[i] = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.BatchNormalization(),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.BatchNormalization(),
            layers.Conv2D(32, kernel_size=(5, 5), activation="relu", strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.BatchNormalization(),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.BatchNormalization(),
            layers.Conv2D(64, kernel_size=(5, 5), activation="relu", strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )
    model[i].compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [4]:
# CREATE MORE IMAGES VIA DATA AUGMENTATION
datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range = 0.10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1)

In [5]:
# Set a learning rate annealer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.00001)

# Configure the training period and batch size
epochs = 45
batch_size = 64

In [6]:
# ENSEMBLE TRAINING
history = [0] * model_count
for i in range(model_count):
    # create validation dataset
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.30, random_state=i)
    datagen.fit(X_train_split, seed=i)
    batches = datagen.flow(X_train_split, y_train_split, batch_size=batch_size)
    val_batches = datagen.flow(X_val_split, y_val_split, batch_size=batch_size)
    # Train the model
    history[i] = model[i].fit(
        batches,
        epochs=epochs,
        steps_per_epoch=len(X_train_split) // batch_size,
        validation_data=val_batches,
        callbacks=[reduce_lr])

Epoch 1/45
459/459 [==============================] - 59s 128ms/step - loss: 0.3150 - accuracy: 0.9009 - val_loss: 0.9894 - val_accuracy: 0.6821
Epoch 2/45
459/459 [==============================] - 55s 121ms/step - loss: 0.1033 - accuracy: 0.9683 - val_loss: 0.0904 - val_accuracy: 0.9730
Epoch 3/45
459/459 [==============================] - 65s 141ms/step - loss: 0.0847 - accuracy: 0.9741 - val_loss: 0.0662 - val_accuracy: 0.9807
Epoch 4/45
459/459 [==============================] - 65s 142ms/step - loss: 0.0727 - accuracy: 0.9793 - val_loss: 0.0838 - val_accuracy: 0.9742
Epoch 5/45
459/459 [==============================] - 64s 139ms/step - loss: 0.0632 - accuracy: 0.9808 - val_loss: 0.0747 - val_accuracy: 0.9772
Epoch 6/45
459/459 [==============================] - 62s 135ms/step - loss: 0.0567 - accuracy: 0.9825 - val_loss: 0.0510 - val_accuracy: 0.9842
Epoch 7/45
459/459 [==============================] - 66s 143ms/step - loss: 0.0561 - accuracy: 0.9841 - val_loss: 0.0540 - val_ac

459/459 [==============================] - 55s 120ms/step - loss: 0.0700 - accuracy: 0.9790 - val_loss: 0.0627 - val_accuracy: 0.9825
Epoch 5/45
459/459 [==============================] - 61s 132ms/step - loss: 0.0597 - accuracy: 0.9814 - val_loss: 0.0793 - val_accuracy: 0.9779
Epoch 6/45
459/459 [==============================] - 71s 154ms/step - loss: 0.0570 - accuracy: 0.9836 - val_loss: 0.0463 - val_accuracy: 0.9859
Epoch 7/45
459/459 [==============================] - 60s 131ms/step - loss: 0.0521 - accuracy: 0.9845 - val_loss: 0.0503 - val_accuracy: 0.9850
Epoch 8/45
459/459 [==============================] - 59s 129ms/step - loss: 0.0515 - accuracy: 0.9846 - val_loss: 0.0529 - val_accuracy: 0.9858
Epoch 9/45
459/459 [==============================] - ETA: 0s - loss: 0.0473 - accuracy: 0.9866
Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
459/459 [==============================] - 55s 120ms/step - loss: 0.0473 - accuracy: 0.9866 - val_loss: 0.0650

459/459 [==============================] - 64s 140ms/step - loss: 0.0554 - accuracy: 0.9831 - val_loss: 0.0533 - val_accuracy: 0.9842
Epoch 8/45
459/459 [==============================] - 63s 136ms/step - loss: 0.0533 - accuracy: 0.9846 - val_loss: 0.0539 - val_accuracy: 0.9845
Epoch 9/45
459/459 [==============================] - 58s 127ms/step - loss: 0.0499 - accuracy: 0.9848 - val_loss: 0.0465 - val_accuracy: 0.9865
Epoch 10/45
459/459 [==============================] - 57s 123ms/step - loss: 0.0452 - accuracy: 0.9865 - val_loss: 0.0691 - val_accuracy: 0.9799
Epoch 11/45
459/459 [==============================] - 60s 131ms/step - loss: 0.0443 - accuracy: 0.9869 - val_loss: 0.0475 - val_accuracy: 0.9855
Epoch 12/45
459/459 [==============================] - ETA: 0s - loss: 0.0405 - accuracy: 0.9877
Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
459/459 [==============================] - 58s 127ms/step - loss: 0.0405 - accuracy: 0.9877 - val_loss: 0.0

459/459 [==============================] - 70s 152ms/step - loss: 0.0481 - accuracy: 0.9862 - val_loss: 0.0535 - val_accuracy: 0.9847
Epoch 11/45
459/459 [==============================] - 62s 136ms/step - loss: 0.0454 - accuracy: 0.9870 - val_loss: 0.0553 - val_accuracy: 0.9829
Epoch 12/45
459/459 [==============================] - 62s 136ms/step - loss: 0.0398 - accuracy: 0.9888 - val_loss: 0.0591 - val_accuracy: 0.9840
Epoch 13/45
459/459 [==============================] - ETA: 0s - loss: 0.0428 - accuracy: 0.9878
Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
459/459 [==============================] - 60s 132ms/step - loss: 0.0428 - accuracy: 0.9878 - val_loss: 0.0541 - val_accuracy: 0.9836
Epoch 14/45
459/459 [==============================] - 69s 150ms/step - loss: 0.0274 - accuracy: 0.9923 - val_loss: 0.0347 - val_accuracy: 0.9893
Epoch 15/45
459/459 [==============================] - 62s 136ms/step - loss: 0.0227 - accuracy: 0.9929 - val_loss: 0

459/459 [==============================] - 55s 121ms/step - loss: 0.0303 - accuracy: 0.9904 - val_loss: 0.0409 - val_accuracy: 0.9886
Epoch 13/45
459/459 [==============================] - 52s 113ms/step - loss: 0.0278 - accuracy: 0.9914 - val_loss: 0.0370 - val_accuracy: 0.9899
Epoch 14/45
459/459 [==============================] - 53s 116ms/step - loss: 0.0306 - accuracy: 0.9915 - val_loss: 0.0343 - val_accuracy: 0.9913
Epoch 15/45
459/459 [==============================] - 54s 117ms/step - loss: 0.0283 - accuracy: 0.9913 - val_loss: 0.0528 - val_accuracy: 0.9860
Epoch 16/45
459/459 [==============================] - 54s 118ms/step - loss: 0.0262 - accuracy: 0.9922 - val_loss: 0.0331 - val_accuracy: 0.9903
Epoch 17/45
459/459 [==============================] - ETA: 0s - loss: 0.0248 - accuracy: 0.9927
Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
459/459 [==============================] - 55s 120ms/step - loss: 0.0248 - accuracy: 0.9927 - val_loss: 0

Epoch 15/45
459/459 [==============================] - 53s 115ms/step - loss: 0.0204 - accuracy: 0.9935 - val_loss: 0.0287 - val_accuracy: 0.9920
Epoch 16/45
459/459 [==============================] - 53s 115ms/step - loss: 0.0184 - accuracy: 0.9942 - val_loss: 0.0364 - val_accuracy: 0.9917
Epoch 17/45
459/459 [==============================] - 53s 115ms/step - loss: 0.0173 - accuracy: 0.9945 - val_loss: 0.0307 - val_accuracy: 0.9924
Epoch 18/45
459/459 [==============================] - 53s 116ms/step - loss: 0.0161 - accuracy: 0.9947 - val_loss: 0.0301 - val_accuracy: 0.9918
Epoch 19/45
459/459 [==============================] - 54s 118ms/step - loss: 0.0145 - accuracy: 0.9955 - val_loss: 0.0382 - val_accuracy: 0.9902
Epoch 20/45
459/459 [==============================] - 53s 115ms/step - loss: 0.0161 - accuracy: 0.9947 - val_loss: 0.0307 - val_accuracy: 0.9928
Epoch 21/45
459/459 [==============================] - 53s 117ms/step - loss: 0.0148 - accuracy: 0.9953 - val_loss: 0.0308 -

459/459 [==============================] - 79s 172ms/step - loss: 0.0197 - accuracy: 0.9939 - val_loss: 0.0372 - val_accuracy: 0.9901
Epoch 20/45
459/459 [==============================] - 77s 168ms/step - loss: 0.0194 - accuracy: 0.9942 - val_loss: 0.0309 - val_accuracy: 0.9920
Epoch 21/45
459/459 [==============================] - 77s 167ms/step - loss: 0.0227 - accuracy: 0.9929 - val_loss: 0.0305 - val_accuracy: 0.9908
Epoch 22/45
459/459 [==============================] - 68s 148ms/step - loss: 0.0175 - accuracy: 0.9944 - val_loss: 0.0325 - val_accuracy: 0.9908
Epoch 23/45
459/459 [==============================] - ETA: 0s - loss: 0.0193 - accuracy: 0.9937
Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
459/459 [==============================] - 68s 149ms/step - loss: 0.0193 - accuracy: 0.9937 - val_loss: 0.0332 - val_accuracy: 0.9910
Epoch 24/45
459/459 [==============================] - 74s 161ms/step - loss: 0.0144 - accuracy: 0.9956 - val_loss: 0

459/459 [==============================] - 69s 150ms/step - loss: 0.0153 - accuracy: 0.9954 - val_loss: 0.0334 - val_accuracy: 0.9914
Epoch 23/45
459/459 [==============================] - ETA: 0s - loss: 0.0160 - accuracy: 0.9950
Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
459/459 [==============================] - 66s 144ms/step - loss: 0.0160 - accuracy: 0.9950 - val_loss: 0.0342 - val_accuracy: 0.9913
Epoch 24/45
459/459 [==============================] - 67s 145ms/step - loss: 0.0116 - accuracy: 0.9966 - val_loss: 0.0307 - val_accuracy: 0.9921
Epoch 25/45
459/459 [==============================] - 70s 151ms/step - loss: 0.0099 - accuracy: 0.9967 - val_loss: 0.0323 - val_accuracy: 0.9918
Epoch 26/45
459/459 [==============================] - ETA: 0s - loss: 0.0104 - accuracy: 0.9964
Epoch 00026: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
459/459 [==============================] - 66s 143ms/step - loss: 0.0104 - accuracy: 0.

Epoch 26/45
459/459 [==============================] - 67s 146ms/step - loss: 0.0146 - accuracy: 0.9958 - val_loss: 0.0264 - val_accuracy: 0.9921
Epoch 27/45
459/459 [==============================] - 64s 139ms/step - loss: 0.0116 - accuracy: 0.9963 - val_loss: 0.0280 - val_accuracy: 0.9934
Epoch 28/45
459/459 [==============================] - 63s 138ms/step - loss: 0.0117 - accuracy: 0.9961 - val_loss: 0.0258 - val_accuracy: 0.9929
Epoch 29/45
459/459 [==============================] - 65s 141ms/step - loss: 0.0121 - accuracy: 0.9962 - val_loss: 0.0263 - val_accuracy: 0.9927
Epoch 30/45
459/459 [==============================] - ETA: 0s - loss: 0.0103 - accuracy: 0.9971
Epoch 00030: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
459/459 [==============================] - 68s 147ms/step - loss: 0.0103 - accuracy: 0.9971 - val_loss: 0.0272 - val_accuracy: 0.9933
Epoch 31/45
459/459 [==============================] - 67s 146ms/step - loss: 0.0087 - accuracy: 0.9972 -

459/459 [==============================] - 69s 151ms/step - loss: 0.0112 - accuracy: 0.9965 - val_loss: 0.0267 - val_accuracy: 0.9928
Epoch 29/45
459/459 [==============================] - 58s 127ms/step - loss: 0.0123 - accuracy: 0.9955 - val_loss: 0.0218 - val_accuracy: 0.9942
Epoch 30/45
459/459 [==============================] - 62s 135ms/step - loss: 0.0099 - accuracy: 0.9970 - val_loss: 0.0255 - val_accuracy: 0.9939
Epoch 31/45
459/459 [==============================] - 100s 217ms/step - loss: 0.0110 - accuracy: 0.9966 - val_loss: 0.0246 - val_accuracy: 0.9943
Epoch 32/45
459/459 [==============================] - ETA: 0s - loss: 0.0096 - accuracy: 0.9967
Epoch 00032: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
459/459 [==============================] - 117s 255ms/step - loss: 0.0096 - accuracy: 0.9967 - val_loss: 0.0240 - val_accuracy: 0.9937
Epoch 33/45
459/459 [==============================] - 83s 182ms/step - loss: 0.0087 - accuracy: 0.9974 - val_loss: 

In [7]:
# ENSEMBLE PREDICTIONS AND SUBMIT
yhats = [mod.predict(test) for mod in model]
yhats = np.array(yhats)
summed = np.sum(yhats, axis=0)
# select the indix with the maximum probability
results = np.argmax(summed, axis=1)

In [8]:
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)